In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import ExtraTreesRegressor,VotingClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn import preprocessing
import scoring as score # para hacer los reportes de puntajes
from time import time

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes

properati = pd.read_csv('datos/properati_final.csv')

<IPython.core.display.Javascript object>

In [12]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['state_name']
le_barrio.fit(barrios)
properati['state_name'] = le_barrio.transform(barrios)

le_zona = preprocessing.LabelEncoder()
zona=properati['place_name']
le_zona.fit(zona)
properati['place_name'] = le_zona.transform(zona)

le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)

In [13]:
#preparo set de datos
X = zip(properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati["property_type"],properati['state_name'],properati['place_name'])
y = properati["price_aprox_usd"]

# GBoost

In [14]:
%%notify

gb= ExtraTreesRegressor(n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

n_estimators=np.arange(10,100,10)

param_grid = {"n_estimators": n_estimators}

search = RandomizedSearchCV(gb, param_distributions=param_grid ,cv=5,n_iter=2) 
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 47.50 segundos para 2 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.990 (std: 0.001)
Promedio validation score: 0.615 (std: 0.135)
Promedio fit time: 2.662s
Hiper-parametros: {'n_estimators': 70}

Puesto: 2
Promedio training score: 0.990 (std: 0.001)
Promedio validation score: 0.606 (std: 0.160)
Promedio fit time: 3.064s
Hiper-parametros: {'n_estimators': 90}



<IPython.core.display.Javascript object>

In [15]:
gboost = search.best_estimator_

# XGBoost

In [17]:
%%notify
from xgboost import XGBRegressor
xgb= XGBRegressor(nthread=-1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

learning_rate=np.arange(0.3,0.9,0.2)

param_grid = {"learning_rate": learning_rate,'min_child_weight':[1,3],'gamma':[i/10.0 for i in range(1,9)],'subsample':[i/10.0 for i in range(6,8)]}

search1 = GridSearchCV(xgb, param_grid=param_grid ,cv=5) 
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


GridSearchCV duro 27.11 segundos para 2 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.989 (std: 0.002)
Promedio validation score: 0.532 (std: 0.313)
Promedio fit time: 2.190s
Hiper-parametros: {'n_estimators': 60}

Puesto: 2
Promedio training score: 0.989 (std: 0.002)
Promedio validation score: 0.499 (std: 0.360)
Promedio fit time: 0.836s
Hiper-parametros: {'n_estimators': 20}



<IPython.core.display.Javascript object>

In [18]:
xgboost = search.best_estimator_

# RF

In [20]:
%%notify

rf = RandomForestRegressor(n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

criterion = ['mse']
max_features =["auto","log2",None]
n_estimators = np.arange(40,100,5)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = RandomizedSearchCV(rf, param_distributions=param_grid ,cv=5,n_iter=2) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

GridSearchCV duro 60.45 segundos para 2 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.931 (std: 0.014)
Promedio validation score: 0.641 (std: 0.169)
Promedio fit time: 3.884s
Hiper-parametros: {'n_estimators': 80, 'max_features': None, 'criterion': 'mse'}

Puesto: 2
Promedio training score: 0.933 (std: 0.019)
Promedio validation score: 0.582 (std: 0.230)
Promedio fit time: 4.540s
Hiper-parametros: {'n_estimators': 90, 'max_features': None, 'criterion': 'mse'}



<IPython.core.display.Javascript object>

In [ ]:
%%notify

rf = RandomForestClassifier(n_jobs=-1)

criterion = ['gini','entropy']
max_features =["auto","log2",None]
n_estimators = np.arange(40,100,5)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"criterion": criterion,"max_features":max_features,"n_estimators":n_estimators}

search = GridSearchCV(rf, param_distributions=param_grid ,cv=5,n_iter=10) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

In [21]:
rf= search.best_estimator_

NameError: name 'mejor2' is not defined

In [9]:
search.score(X_test,y_test)

0.86392637136990325

# ETC

# MLP

In [22]:
from sklearn.neural_network import MLPRegressor

In [23]:
%%notify

mlp = MLPRegressor()

activation = ["identity", "logistic", "tanh", "relu"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

param_grid = {"activation": activation}

search = GridSearchCV(mlp, param_grid=param_grid ,cv=5,n_jobs=-1) #refit deja el estimador fiteado con los mejores hiperparametros
start = time()
search.fit(X_train, y_train)

print("GridSearchCV duro %.2f segundos para %d candidatos a hiper-parametros."
    % (time() - start, len(search.cv_results_['params'])))
print("")
score.report_single(search.cv_results_)

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV duro 804.48 segundos para 4 candidatos a hiper-parametros.

Puesto: 1
Promedio training score: 0.493 (std: 0.072)
Promedio validation score: -0.121 (std: 1.035)
Promedio fit time: 105.808s
Hiper-parametros: {'activation': 'relu'}

Puesto: 2
Promedio training score: -0.527 (std: 0.059)
Promedio validation score: -0.576 (std: 0.140)
Promedio fit time: 93.532s
Hiper-parametros: {'activation': 'tanh'}

Puesto: 3
Promedio training score: -0.528 (std: 0.059)
Promedio validation score: -0.577 (std: 0.141)
Promedio fit time: 219.882s
Hiper-parametros: {'activation': 'logistic'}

Puesto: 4
Promedio training score: 0.161 (std: 0.131)
Promedio validation score: -0.904 (std: 2.015)
Promedio fit time: 41.417s
Hiper-parametros: {'activation': 'identity'}



/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


<IPython.core.display.Javascript object>

In [24]:
mlp=search.best_estimator_

In [61]:
search.score(X_test,y_test)

0.79543121149897333

# Lineales

In [40]:
from sklearn.linear_model import LogisticRegression

In [ ]:
%%notify

logr = LogisticRegression(n_jobs=-1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

logr.fit(X_train, y_train)
logr.score(X_test,y_test)

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


# Ensamble

In [ ]:
from mlxtend.regressor import StackingRegressor

stregr = StackingRegressor(regressors=[xgb,gboost], 
                           meta_regressor=rf)

In [ ]:
%%notify
stregr.fit(X_train,y_train)

In [ ]:
stregr.score(X_test,y_test)

In [ ]:
errores = stregr.predict(X_test)-y_test
print ("Error maximo:{0}\nError minimo:{1}".format( max(abs(errores)),min(abs(errores))))
print(errores)

In [ ]:
count_max=0
max_error=1000
lista=[]
for error in errores:
    if abs(error)>max_error:
        count_max+=1
        lista.append(abs(error))
print ("Hay {0} de {1} que tienen mas de {2} USD de error ({3}%)".format(count_max,len(errores),max_error,100 * count_max/len(errores)))

In [ ]:
# the histogram of the data
plt.figure(figsize=(12,8))
plt.hist(errores, 100, facecolor='blue')
plt.xlabel('Errores')
plt.ylabel('Cantidad')
#plt.xlim(-10000000, 10000000) #para variar el "zoom a 0"
plt.yscale('log')
plt.show()